In [ ]:
# %% Calculus 2 - Section 14.93
#    Code challenge: multivariable definite integrals

# This code pertains to a calculus course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/pycalc2_x
# The code in this repository is developed to solve the exercises provided along
# the course, and it has been written partially indepentently and partially
# from the code developed by the course instructor.


In [6]:
import numpy                as np
import sympy                as sym
import matplotlib.pyplot    as plt
import matplotlib.colors    as mcolors
import scipy.integrate      as spi
import math
import mpmath
import plotly.graph_objects as go
import sympy.stats
import matplotlib.patches   as patches

from scipy.signal                     import find_peaks
from scipy                            import stats
from IPython.display                  import display,Math
from google.colab                     import files
from IPython.display                  import Audio
from scipy.io                         import wavfile
from mpl_toolkits.mplot3d             import Axes3D
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')


In [ ]:
# %% Exercise 0a
#    Create the functions in both sympy and numpy

# Sympy
x    = sym.symbols('x')
y    = sym.symbols('y')
f_xy = 10 - ((x**2-y**2)/8)

# Lambdify
f_xy_l = sym.lambdify((x,y),f_xy)

# Discretise and get resolution
xx = np.linspace(-10,10,51)
yy = np.linspace(-10,10,51)

dx = xx[1] - xx[0]
dy = yy[1] - yy[0]

# Points grid
XX,YY = np.meshgrid(xx,yy)
Z     = f_xy_l(XX,YY)

# Plot
phi = (1 + np.sqrt(5)) / 2
_,ax = plt.subplots(1,figsize=(phi*5,5))

ax.imshow(Z,extent=[xx[0], xx[-1], yy[-1], yy[0]],aspect='auto',cmap='jet')

ax.set(xlabel='X',ylabel='Y')
ax.set_title(r'$f(x,y) = %s$' %sym.latex(f_xy))

plt.savefig('fig7_codechallenge_93_exercise_0a.png')
plt.show()
files.download('fig7_codechallenge_93_exercise_0a.png')


In [ ]:
# %% Exercise 0b
#    Compute definite integral with sympy

# Bounds
x_a,x_b = 0,5
y_a,y_b = 0,4

# Partial over x (area along x changing as function of y) and full integral
area_x = sym.integrate(f_xy,(x,x_a,x_b))
area   = sym.integrate(f_xy,(x,x_a,x_b),(y,y_a,y_b))

display(Math('%s = %s' %(sym.latex(sym.Integral(f_xy,(x,x_a,x_b),y)),sym.latex(area_x)))), print()
display(Math('%s = %s = %s' %(sym.latex(sym.Integral(f_xy,(x,x_a,x_b),(y,y_a,y_b))),sym.latex(area),sym.latex(area.evalf())))), print()


In [ ]:
# %% Exercise 0c
#    Compute definite integral with sympy

# Plot again with a box around integral bounds
phi = (1 + np.sqrt(5)) / 2
_,ax = plt.subplots(1,figsize=(phi*5,5))

ax.imshow(Z,extent=[xx[0], xx[-1], yy[-1], yy[0]],aspect='auto',cmap='jet')

rect = patches.Rectangle((x_a, y_a), x_b-x_a, y_b-y_a, linewidth=2,edgecolor='r',linestyle=':',facecolor='none')
ax.add_patch(rect)

ax.set(xlabel='X',ylabel='Y')
ax.set_title(r'$f(x,y) = %s$' %sym.latex(f_xy))

plt.savefig('fig8_codechallenge_93_exercise_0c.png')
plt.show()
files.download('fig8_codechallenge_93_exercise_0c.png')

# Integral (get closest indices to the bounds)
x_a_idx = np.argmin(np.abs(xx-x_a))
x_b_idx = np.argmin(np.abs(xx-x_b))
y_a_idx = np.argmin(np.abs(yy-y_a))
y_b_idx = np.argmin(np.abs(yy-y_b))

# Get function surface from bounds
Z_sub = Z[y_a_idx:y_b_idx,x_a_idx:x_b_idx]

print(f'Size of function landscape: {Z.shape}')
print(f'Size of function subset: {Z_sub.shape}')
print()

# Sum (and scale)
area_x_np = np.sum(Z_sub,axis=1) * dx
area_np   = np.sum(Z_sub) * dx * dy

display(Math('%s \\approx %s' %(sym.latex(sym.Integral(f_xy,(x,x_a,x_b),y)),sym.latex(area_x_np)))), print()
display(Math('%s \\approx %s' %(sym.latex(sym.Integral(f_xy,(x,x_a,x_b),(y,y_a,y_b))),sym.latex(area_np))))


In [ ]:
# %% Exercise 0d
#    Compute definite integral with sympy

# The function scipy.integrate.dblquad needs the variables swapped (y,x); kudos
# to python as usual for the chaos
def f_xy_sp(y,x):
    return 10 - (x**2 - y**2) / 8

# Calculate the double integral (inner variable first)
area_sp,_ = spi.dblquad( f_xy_sp,
                         x_a,
                         x_b,
                         lambda x: y_a,
                         lambda x: y_b )

display(Math('%s \\approx %s' %(sym.latex(sym.Integral(f_xy,(x,x_a,x_b),(y,y_a,y_b))),sym.latex(area_sp))))
